# Automating Pattern Selection

In this notebook we will take a look under the hood of Avicenna with the aim of automating it's pattern selection. What we mean by this in detail, which methods we will use and what challenges we will face is part of this development journey. I will try to set up an example (probably the Calculator) and step by step disassemble Avicenna until the point where the pattern selection is needed. I hope to then develop possible solutions to automate the process.

## Calculator Example

### Program under test

In [11]:
import math

def calculator(inp: str) -> float:
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [12]:
from avicenna.avicenna import OracleResult

def oracle(inp: str) -> OracleResult:
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING
    return OracleResult.PASSING

In [13]:
import string

grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"],
}

In [14]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)', 'sqrt(-3)']

### Using Avicenna

In [15]:
from avicenna.avicenna import Avicenna

avicenna = Avicenna(
    grammar,
    oracle,
    initial_inputs,
)

In [16]:
from avicenna.input import Input
from typing import Set

In [17]:
new_inputs: Set[Input] = avicenna.all_inputs.union(avicenna.generate_more_inputs())

In [18]:
from avicenna.monads import Exceptional, check_empty

def loop(test_inputs: Set[Input]):
    test_inputs = avicenna.construct_inputs(test_inputs)
    exclusion_non_terminals = avicenna.learn_relevant_features()

    new_candidates = avicenna.pattern_learner.learn_failure_invariants(
        test_inputs,
        avicenna.precision_truth_table,
        avicenna.recall_truth_table,
        exclusion_non_terminals,
    )

    new_candidates = new_candidates.keys()

    avicenna.best_candidates = new_candidates
    new_inputs = (
        Exceptional.of(lambda: new_candidates)
        .map(avicenna.add_negated_constraints)
        .map(avicenna.generate_inputs)
        .bind(check_empty)
        .recover(avicenna.generate_inputs_with_grammar_fuzzer)
        .reraise()
        .get()
    )
    # LOGGER.info(f"Generated {len(new_inputs)} new inputs.")
    return new_inputs

In [19]:
while avicenna.do_more_iterations():
    new_inputs = loop(new_inputs)

In [20]:
diagnosis = avicenna.finalize()

In [21]:
from isla.language import ISLaUnparser

print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")

print(ISLaUnparser(diagnosis[0]).unparse())
print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")

Avicenna determined the following constraints to describe the failure circumstances:

(exists <function> elem in start:
   (= elem "sqrt") and
exists <maybe_minus> elem_0 in start:
  (= elem_0 "-"))
Avicenna calculated a precision of 100.00% and a recall of 100.00%


- Idee: simuliere den Durchlauf von Avicenna bis zu dem Punkt, an dem die Pattern Selektion relevant ist
- Schnittstelle von der ich Daten bekomme: 
- Schnittstelle an die ich Daten gebe: pattern_learner Z272 get_candidates -> generate_candidates